In [1]:
from sklearn.feature_extraction.text import CountVectorizer # tf-idf
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer # tf-idf
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import Pipeline
from nltk.corpus import stopwords # preprocessing\
import math
from numpy import linalg as LA
import numpy as np
class Engine:
    def __init__(self):
        self.cosine_score = []
        self.train_set = []  # Documents
        self.test_set = []  # Query

    def addDocument(self, word): # fungsi untuk menambahkan dokumen dataset ke dalam list train_set
        self.train_set.append(word)

    def setQuery(self, word):  # fungsi untuk menambahkan data query ke dalam list test_Set
        self.test_set.append(word)

    def process_score(self):
        vectorizer = TfidfVectorizer(stop_words='english', max_features= 1000, max_df = 0.5, smooth_idf=True)
        svd_model = TruncatedSVD(n_components=100,algorithm='randomized',n_iter=10)
        lsa = Pipeline([('tfidf', vectorizer),('svd', svd_model)])
        #print(lsa)
        #transformer = TfidfTransformer(stop_words='english', use_idf=True, smooth_idf=True)

        trainVectorizerArray = lsa.fit_transform(self.train_set).tolist() 
        # menghitung Bobot dokumen dataset dan uji dan kemudian disimpan dalam bentuk array 
        testVectorizerArray = lsa.transform(self.test_set).tolist()

        cx = lambda a, b: round(np.inner(a, b) / (LA.norm(a) * LA.norm(b)), 3) 
        #fungsi tanpa nama untuk normalisasi data dan definisi rumus Cosine Similarity 
        #print (testVectorizerArray)
        output = []
        for i in range(0, len(testVectorizerArray)):
            output.append([])

        for vector in trainVectorizerArray:
            #print (vector)
            u = 0
            for testV in testVectorizerArray:
                #perhitungan Cosine Similarity dalam bentuk vector dari dataset dengan query
                #yang di masukan yang kemudian mengembalikan nilai cosine ke dalam variable
                #cosine_score dalam bentuk list.
                #print (testV)
                cosine = cx(vector, testV)
                #print(cosine)
                #                 self.cosine_score.append(cosine)
                #                 bulatin = (round(cosine),2)
                if math.isnan(cosine):
                    cosine = 0
                output[u].append((cosine))
                u = u + 1
        return output
        #return testVectorizerArray

import pandas as pd
setD = pd.read_excel('preprocessed-datasett.xlsx',sep=',')
setD = setD.dropna(axis=0, how='any')
#print(setD)
Doc = setD['preprocessed_judul'].astype('str')

#print(Doc)
kunci = pd.read_excel('data1-100.xlsx',sep=',')
kunci = kunci.dropna(axis=0, how='any')
query = kunci['preprocessed_judul']
query = query.replace('NaN', 0)
query = ['text categorization many redundant features using aggressive feature selection make svms competitive c4 5']
engine = Engine()

docs = [str(x) for x in Doc]
documentNames = list()

for i, doc in enumerate(docs):
    engine.addDocument(doc) 
    documentNames.append("Document_{}".format(i+1))
for queries in query:
    engine.setQuery(queries) #inputandata uji

titles_score = engine.process_score()
ScoreDf = (pd.DataFrame(titles_score)).T
ScoreDf.columns = query
ScoreDf["Documents"] = documentNames
ScoreDf["Judul"] = setD["Judul"].values
ScoreDf["Abstrak"] = setD["preprocessed_abstrak"].values
#ScoreDf


df_listed = list()
for i in query:
    labels = list()
    for j in ScoreDf[i]:
        if j>0.000:
            labels.append(1)
        else:
            labels.append(0)
    datadf = pd.DataFrame(ScoreDf[i])
    datadf['Documents'] = ScoreDf['Documents']
    datadf['Labels'] = labels
    datadf['abstrak'] = setD['preprocessed_abstrak'].values
    datadf['reviewer'] = setD['Reviewer'].values
    datadf['Judul'] = ScoreDf['Judul'].values
    df_listed.append(datadf.sort_values(by=[i], ascending=False))
df_listed[0].head(15)



C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:32: RuntimeWarning: invalid value encountered in double_scalars


,text categorization many redundant features using aggressive feature selection make svms competitive c4 5,Documents,Labels,abstrak,reviewer,Judul
5400,0.880,Document_5401,1,text categorization important research area ma...,wei-ying ma,ocfs: optimal orthogonal centroid feature sele...
9389,0.854,Document_9390,1,text span forming either single word series tw...,ariel fuxman,smart selection of text spans
12904,0.820,Document_12905,1,improved system method provided feature select...,anirban dasgupta,system and method of feature selection for tex...
6348,0.801,Document_6349,1,recognizing text character natural scene image...,ying li,feature representations for scene text charact...
5383,0.760,Document_5384,1,music classification technique computes histog...,tao li,music feature extraction using wavelet coeffic...
8491,0.758,Document_8492,1,although human hand complex biomechanical syst...,eric xing,feature selection for grasp recognition from o...
3377,0.757,Document_3378,1,paper describes several known new methods feat...,dunja mladenic,feature subset selection in text-learning
9184,0.757,Document_9185,1,frequent problem density levelset estimation c...,ulf brefeld,feature selection for density level-sets
4357,0.757,Document_4358,1,feature selection preprocessing step machine l...,huan liu,feature selection with selective sampling
4410,0.756,Document_4411,1,discretization turn numeric attributes discret...,huan liu,feature selection via discretization
